# Nutrition Calculator  - Australian Food Composition Database

mission statement

#### Data sources
nutrition data: https://www.foodstandards.gov.au/science-data/monitoringnutrients/afcd
recommended intake data: https://www.myfooddata.com/articles/recommended-daily-intakes.php

To Do:
- Make units kiss
    - Check certain foods
- Add omega 6s properly
- Add nutrients back in with improved data imputation
- Deal with units and percentage with regex extract to get rid of duplicates and solve above
- Use classifier trained on recipe file to make appropraite food portions
- Do dot product of foods with DV
    - Adjust weights on nutrients

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
sns.set()
%matplotlib inline

In [2]:
# Regex debugger #
pattern = r'\/gN'
df = pd.DataFrame({ 'Lysine \n(mg/gN)': [420, 69], 'Lysine \n(mg)':[1,2]})
new_cols = df.columns.str.contains(pattern)
print(new_cols)

[ True False]


In [3]:
# Import food and nutrition data #

food = pd.read_excel('Australian_Food_Composition_Database/Food Details.xlsx') 
nutrient = pd.read_excel('Australian_Food_Composition_Database/Nutrient file.xlsx',
                         sheet_name='All solids & liquids per 100g')

## Need to drop percent fatty acid content, not necessary for calculation and data cleaning causes repition ##
nutrient = nutrient.drop(columns = ['Total polyunsaturated fatty acids, equated (%T)', 
                          'Total long chain omega 3 fatty acids, equated \n(%T)'])
## Import Daily Value (DV) ##

dv = pd.read_csv('daily value.csv', index_col=0)
dv.head()

,RDI,DV,UL
Nutrient,,,
Calories,2000,2000,NaN
Fat,NaN,78g,NaN
Saturated Fat,NaN,20g,NaN
Cholesterol,NaN,300mg,NaN
Carbs,130g,275g,NaN


In [4]:
## Need to drop percent content columns. Not relevant to goal and causing duplicate columns later on ##
print("Number of columns in df before drop:", len(nutrient.columns))

percent_pattern = r'\(%\w+\)'
percent_cols = nutrient.columns.str.contains(percent_pattern)
print("Number of columns with % in header:", percent_cols.sum())
nutrient = nutrient.drop(columns = nutrient.columns[percent_cols])
print("Number of columns after dropping %:", len(nutrient.columns))

gN_pattern = r'\/gN'
gN_cols = nutrient.columns.str.contains(gN_pattern)
print("Number of columns with /gN in header:", gN_cols.sum())
nutrient = nutrient.drop(columns = nutrient.columns[gN_cols])
print("Number of columns after dropping /gN:", len(nutrient.columns))

Number of columns in df before drop: 291
Number of columns with % in header: 69
Number of columns after dropping %: 222
Number of columns with /gN in header: 18
Number of columns after dropping /gN: 204


In [5]:
nutrient.head()

,Public Food Key,Classification,Food Name,"Energy with dietary fibre, equated \n(kJ)","Energy, without dietary fibre, equated \n(kJ)",Moisture (water) \n(g),Protein \n(g),Nitrogen \n(g),"Fat, total \n(g)",Ash \n(g),...,Leucine \n(mg),Lysine \n(mg),Methionine \n(mg),Phenylalanine \n(mg),Proline \n(mg),Serine \n(mg),Threonine \n(mg),Tyrosine \n(mg),Tryptophan \n(mg),Valine \n(mg)
0,F002258,31302.0,"Cardamom seed, dried, ground",1236,1012,8.3,10.8,1.72,6.7,5.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,155,NaN
1,F002893,31302.0,"Chilli (chili), dried, ground",1280,1002,10.8,13.4,2.14,14.3,11.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,69,NaN
2,F002963,31302.0,"Cinnamon, dried, ground",1004,579,10.6,4.0,0.64,1.2,3.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,49,NaN
3,F002970,31302.0,"Cloves, dried, ground",1389,1118,9.9,6.0,0.96,13.0,5.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30,NaN
4,F003190,31302.0,"Coriander seed, dried, ground",1344,1009,8.9,12.4,1.98,17.8,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,178,NaN


## Data Cleaning

In [6]:
# Get rid of \n's from headers #
col_pattern = r'\s?\n(.+)'
nutr_cols = nutrient.columns.str.replace(col_pattern, '')
new_cols = dict(zip(nutrient.columns, nutr_cols))
nutrient_clean = nutrient.rename(columns=new_cols)

# Change energy to calories #

fiber=True #toggle this to change how calories from fiber are calculated

if fiber==True:
    energy_cols = {'Energy with dietary fibre, equated': 'Calories'}
    nutrient_clean = nutrient_clean.rename(columns=energy_cols)
    nutrient_clean = nutrient_clean.drop(columns='Energy, without dietary fibre, equated')

else:
    energy_cols = {'Energy, without dietary fibre, equated': 'Calories'}
    nutrient_clean = nutrient_clean.rename(columns=energy_cols)
    nutrient_clean = nutrient_clean.drop(columns='Energy with dietary fibre, equated')

## Convert kJ to kCal ## 
nutrient_clean['Calories'] = nutrient_clean['Calories'].apply(lambda x: x/4.2)  #multiply kJ by this to get kcal

In [7]:
# Extract Units From Cols#

### Attempt 1 - Deleting###
"""The main plan is to delete units in table header. Because every unit in both datasets are metric, an inconsistency 
in units would manifest as a 3 orders of magnitude off which will be investigated in further analysis."""

# Get rid of (abbrev.)#
col_pattern = r' \(.+\)'
nutr_cols = nutrient_clean.columns.str.replace(col_pattern, '')
new_cols = dict(zip(nutrient_clean.columns, nutr_cols))
nutrient_clean = nutrient_clean.rename(columns=new_cols)


In [8]:
# Need to change column names to be equivalent between food and DV #

## Vitamins ##
vitamins = {'Thiamin': 'Vitamin B1',
'Riboflavin': 'Vitamin B2',
'Niacin derived equivalents': 'Vitamin B3', 
'Pantothenic acid':'Vitamin B5',
'Pyridoxine':'Vitamin B6',
'Biotin': 'Vitamin B7',
'Total folates': 'Vitamin B9',
'Cobalamin': 'Vitamin B12',
'Vitamin A retinol equivalents':'Vitamin A',
'Vitamin D3 equivalents': 'Vitamin D'
           }
nutrient_clean = nutrient_clean.rename(columns=vitamins)

## Fat ##
nutrient_clean = nutrient_clean.rename(columns = {'Fat, total': 'Fat', 
                                                    'Total saturated fatty acids, equated': 'Saturated Fat',
                                                   'Total long chain omega 3 fatty acids, equated': 'Omega 3s'
                                                   })

### Adding omega 6s from manual Calc ###
### Assuming any polyunsaturated fat that isn't omega-3 is an omega-6
nutrient_clean['Omega 6s'] = round(nutrient_clean['Total polyunsaturated fatty acids, equated'] - nutrient_clean['Omega 3s']*0.001,2)

## Carbohydrates ##
carbs = {'Total sugars':'Sugar',
          'Total dietary fibre':'Fiber',
        'Available carbohydrate, with sugar alcohols':'Carbs'}
nutrient_clean = nutrient_clean.rename(columns=carbs)

## Misc Nutrients ##

misc_nutr = {'Cystine plus cysteine':'Cystine',
            }
nutrient_clean = nutrient_clean.rename(columns=misc_nutr)


### Investigating NaN's

In [9]:
# Look at nutrient NaNs and determine whether or not they represent 0's

## Using leucine as an example 
leucine = nutrient_clean['Leucine']
print('Number of foods with null leucine values:', leucine.isnull().sum(),
      '\nNumber of foods with non-null:', leucine.notnull().sum())

notnull_leucine = nutrient_clean[leucine.notnull()]
null_leucine = nutrient_clean[leucine.isnull()]

"""The vast majority of foods are missing amino acid data. It would require combining an entire dataset which would
involve matching food names accurately, which is outside of the scope of this project. Either food with missing data 
must be excluded, or we must exclude these nutrients from the fitting criteria. In order to preserve the most data
possible, I an doing to drop the nutrients instead of the foods. For similar reasons as listed below, amino acids
are subcategories of protein, which is included in the data set. I do believe the fitting of these nutrients, like
many others being excluded, to be important and should be subject of an expansion to this study."""

# Make NaNs in food data 0 #
# nutrient_values_only = nutrient_values_only.fillna(0)


## Find columns with mostly NaN's and drop

mostly_empty = []

for cols in nutrient_clean:
    if nutrient_clean[cols].isnull().sum() > nutrient_clean[cols].notnull().sum():
        mostly_empty.append(cols)
        
nutrient_clean = nutrient_clean.drop(columns=mostly_empty)

Number of foods with null leucine values: 1435 
Number of foods with non-null: 181


In [10]:
def shared_columns(df1, df2):
    
    """Finds which columns in df1 are and are not shared with df2"""
    
    df1_cols = df1.columns
    df2_cols = df2.columns
    
    shared_cols = []
    diff_cols = []
    
    for cols in df1_cols:
        if cols in df2_cols:
            shared_cols.append(cols)
        else:
            diff_cols.append(cols)
    
    return shared_cols, diff_cols

In [11]:
## Find DV columns that aren't equal to nutrient cols ##

dv_clean = dv.rename(index={'Lutein+zeazanthin': 'Lutein'})
nutr_cols = nutrient_clean.columns
dv_shared = shared_columns(dv_clean.transpose(), nutrient_clean)


"""This data does not exist in the food list df, therefore we will ignore these nutrients. Water is a 
nutrient that should be satisfied outside of food consumption. Therefore, it should be excluded
from the analysis. Furthermore, the remaining incongruent sets are non-essential amino acids which can
be produced in the human body, making it less important for consumption. Vitamin K and Choline
are essential in the human diet. However, the food data set does not have this included. This data could 
be imputed in the event of expanding the model for a more robust study."""

dv_clean = dv_clean.drop(index=dv_shared[1])
print("Dropping columns:", dv_shared[1])
dv_shared_dropped = shared_columns(dv_clean.transpose(), nutrient_clean)

print("Shared columns:", len(dv_shared_dropped[0]),"\nDifferent columns:", len(dv_shared_dropped[1]))



Dropping columns: ['Vitamin K', 'Choline', 'Lycopene', 'Lutein', 'Water', 'Galactose', 'Histidine', 'Threonine', 'Isoleucine', 'Lysine', 'Leucine', 'Methionine', 'Cystine', 'Phenylalanine', 'Tyrosine', 'Valine', 'Arginine', 'Alanine', 'Aspartic Acid', 'Glutamic Acid', 'Glycine', 'Proline', 'Serine', 'Hydroxyproline']
Shared columns: 40 
Different columns: 0


In [12]:
# Find nutrient columns that aren't in the DV columns #

nutr_cols = nutrient_clean.columns
nutr_shared = shared_columns(nutrient_clean, dv_clean.transpose())
print("Shared columns:", len(nutr_shared[0]),"\nDifferent columns:", len(nutr_shared[1]))

"""Much of the remaining dataset is anti-nutrients, subcategories of nutrients which are encapsulated by other
nutrients, and nutrients which have no recommended value. Currently, anti-nutrients are not in the scope of this
study but could be a valuable addition later. Therefore, I will drop all excess columns except the food name
column."""

food_name = nutrient_clean['Food Name']
print("Dropping excess nutrient columns", nutr_shared[1])

nutrient_values_only = nutrient_clean.drop(columns=nutr_shared[1])
nutr_shared = shared_columns(dv_clean.transpose(), nutrient_values_only)
print("Shared columns:", len(nutr_shared[0]),"\nDifferent columns:", len(nutr_shared[1]))
# print(nutr_shared[1])

Shared columns: 40 
Different columns: 47
Dropping excess nutrient columns ['Public Food Key', 'Classification', 'Food Name', 'Moisture', 'Nitrogen', 'Alcohol', 'Added sugars', 'Free sugars', 'Available carbohydrate, without sugar alcohols', 'Retinol', 'Alpha-carotene', 'Beta-carotene', 'Cryptoxanthin', 'Beta-carotene equivalents', 'Niacin', 'Niacin derived from tryptophan', 'Folate, natural', 'Folic acid', 'Dietary folate equivalents', 'Cholecalciferol', 'Ergocalciferol', '25-hydroxy cholecalciferol', '25-hydroxy ergocalciferol', 'Alpha tocopherol', 'C14', 'C15', 'C16', 'C17', 'C18', 'C22', 'C14:1', 'C16:1', 'C18:1', 'Total monounsaturated fatty acids, equated', 'C18:2w6', 'C18:3w3', 'C18:3w6', 'C20:2w6', 'C20:3w6', 'C20:4w6', 'C20:5w3', 'C22:5w3', 'C22:4w6', 'C22:6w3', 'Total polyunsaturated fatty acids, equated', 'Total trans fatty acids, imputed', 'Caffeine']
Shared columns: 40 
Different columns: 0


In [13]:
np.asarray(nutrient_clean.columns)

array(['Public Food Key', 'Classification', 'Food Name', 'Calories',
       'Moisture', 'Protein', 'Nitrogen', 'Fat', 'Ash', 'Fiber',
       'Alcohol', 'Fructose', 'Glucose', 'Sucrose', 'Maltose', 'Lactose',
       'Sugar', 'Added sugars', 'Free sugars', 'Starch',
       'Available carbohydrate, without sugar alcohols', 'Carbs',
       'Calcium', 'Copper', 'Iodine', 'Iron', 'Magnesium', 'Manganese',
       'Phosphorus', 'Potassium', 'Selenium', 'Sodium', 'Zinc', 'Retinol',
       'Alpha-carotene', 'Beta-carotene', 'Cryptoxanthin',
       'Beta-carotene equivalents', 'Vitamin A', 'Vitamin B1',
       'Vitamin B2', 'Niacin', 'Niacin derived from tryptophan',
       'Vitamin B3', 'Vitamin B5', 'Vitamin B6', 'Vitamin B12',
       'Folate, natural', 'Folic acid', 'Vitamin B9',
       'Dietary folate equivalents', 'Vitamin C', 'Cholecalciferol',
       'Ergocalciferol', '25-hydroxy cholecalciferol',
       '25-hydroxy ergocalciferol', 'Vitamin D', 'Alpha tocopherol',
       'Vitamin E', 'C14

In [14]:
# Extract Units From DV #

unit_pattern = r'\d+(\w+)'
number_pattern = r'(\d+(.?\d+))'
dv_clean['RDI units'] = dv_clean['RDI'].str.extract(unit_pattern)
dv_clean['RDI'] = dv_clean['RDI'].str.extract(number_pattern)
dv_clean['DV units'] = dv_clean['DV'].str.extract(unit_pattern)
dv_clean['DV'] = dv_clean['DV'].str.extract(number_pattern)

# Make numeric entries numeric type
dv_clean['DV'] = pd.to_numeric(dv_clean['DV'])
dv_clean['RDI'] = pd.to_numeric(dv_clean['RDI'])

In [20]:
dv_clean['RDI'].isnull().sum()

17

In [21]:
dv_clean['DV'].isnull().sum()

12

In [34]:
type(dv_clean['DV'])

pandas.core.series.Series

In [37]:
df = pd.DataFrame({'DV': dv_clean['DV'], 'RDI':dv_clean['RDI']})
# df = df.transpose()
df

,DV,RDI
Nutrient,,
Calories,2000.0,2000.0
Fat,78.0,NaN
Saturated Fat,20.0,NaN
Cholesterol,300.0,NaN
Carbs,275.0,130.0
Fiber,28.0,38.0
Sugar,50.0,NaN
Protein,50.0,56.0
Vitamin A,3000.0,900.0


In [49]:
"""Let's combine the entries in RDI and DV to create a daily value with few NaNs, taking the highest value"""

dv_col = dv_clean['DV'] 
rdi_col = dv_clean['RDI']
dv_rdi_combined = []

for dv, rdi in zip(dv_col,rdi_col):
    if dv == np.nan and rdi != np.nan:
        dv_rdi_combined.append(rdi)
    
    elif dv != np.nan and rdi == np.nan:
        dv_rdi_combined.append(dv)
    
    elif dv == np.nan and rdi == np.nan:
        dv_rdi_combined.append(dv)
        
    elif dv != np.nan and rdi != np.nan:
        dv_rdi_combined.append(max(dv,rdi))
        
dv_clean['DV and RDI, Highest'] = dv_rdi_combined

In [50]:
dv_clean['DV and RDI, Highest']

Nutrient
Calories         2000.0
Fat                78.0
Saturated Fat      20.0
Cholesterol       300.0
Carbs             275.0
Fiber              38.0
Sugar              50.0
Protein            56.0
Vitamin A        3000.0
Vitamin B1          1.2
Vitamin B2          1.3
Vitamin B3         16.0
Vitamin B5          NaN
Vitamin B6          1.7
Vitamin B9        400.0
Vitamin B12         2.4
Vitamin C          90.0
Vitamin D          20.0
Vitamin E          15.0
Calcium          1300.0
Copper              0.9
Iodine              NaN
Iron               18.0
Magnesium         420.0
Manganese           2.3
Phosphorus       1250.0
Potassium        4700.0
Selenium           55.0
Sodium           2300.0
Zinc               11.0
Ash                 NaN
Starch              NaN
Sucrose             NaN
Glucose             NaN
Fructose            NaN
Lactose             NaN
Maltose             NaN
Omega 3s            NaN
Omega 6s            NaN
Tryptophan          NaN
Name: DV and RDI, Highest, dtyp

### Sanity Check

In [17]:
a = {}
for nut in nutrient_clean.columns:
    if nut in a:
        a[nut] +=1
    else:
        a[nut] = 1

In [18]:
b = 0
for keys, vals in a.items():
    if vals > 1:
        print('Repeat column')
        b+=1
if b == 0:
    print('No repeat columns')

No repeat columns


## Using linear programming to generate a set of feasible solutions

## Using least squares to find solutions to the linear equation

The tables above leave us with a linear equation of the form:
$$a_{1}f_{1} + ... + a_{m}f_{m} = v_{T}$$ 
where, $$a_{k} \in \mathbb{R}, f_{k} \in V$$ 
and, $$k \in {0,...,m}$$

Here the daily value required is represented by the vector $v_T$, with entries in the vector being daily value nutrient limits.

The list of vectors, $f$, represent the nutrient profile of a given food, with its $k^{th}$ element being the $k^{th}$ nutrient in $v_T$.

The scalars, $a$, are the amounts, or portion size, of the foods in $f$ that satisfy the equation above. This equation can be written in its matrix form:

$$Fa = v_T$$

By solving the equation for the amounts $a$, we can get a list of foods that satisfy our days nutrition needs. Thus, satisfying the goal of this project.

However, because this matrix is not square and therefore not invertible, we must use a least sqaures method of solving the equation which minimizes the following:

$$|v_T - Fa|$$

To do this, we can use least squares method from scipy linalg package. 

In [19]:
## Solve ##
from scipy.linalg import lstsq
rdi_limit = np.asarray(dv_clean['RDI'])
food_matrix = np.asarray(nutrient_values_only.transpose())

food_solution = lstsq(a=food_matrix, b=rdi_limit)
food_solution

ValueError: array must not contain infs or NaNs